In [144]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_theme(palette = "Dark2")
# my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
#   (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
from sklearn.preprocessing import LabelEncoder

In [114]:
import importlib
import Model_functions
importlib.reload(Model_functions)
from Model_functions import Model_class 

In [130]:
df = pd.read_csv("df_127_columns.csv")

In [116]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [137]:
reduce_memory = low_cardinality_cat_cols + ["mode_size","mode_item_color"]

In [131]:
# df["size"] = df["size"].astype("category")
# memory = tuple(df.memory_usage(deep=True))
	
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531170 entries, 0 to 531169
Columns: 127 entries, order_item_id to delivery_time_user_id_mad
dtypes: float64(54), int64(62), object(11)
memory usage: 825.7 MB


In [147]:
df["order_id"] = df["order_id"].astype("category")
df["order_date"] = pd.to_datetime(df["order_date"], format="%Y-%m-%d")
df["delivery_date"] = pd.to_datetime(df["delivery_date"], format="%Y-%m-%d")
df["user_reg_date"] = pd.to_datetime(df["user_reg_date"], format="%Y-%m-%d")
df["user_dob"] = pd.to_datetime(df["user_dob"], format="%Y-%m-%d")
LE = LabelEncoder()
for i in reduce_memory:
    df[i] = LE.fit_transform(df[i])

In [148]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531170 entries, 0 to 531169
Columns: 127 entries, order_item_id to delivery_time_user_id_mad
dtypes: category(1), datetime64[ns](4), float64(54), int32(6), int64(62)
memory usage: 515.1 MB


In [ ]:
# How to shift the date
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)

In [150]:
df_group = df[["order_date","user_id"]].drop_duplicates().sort_values(by=["user_id","order_date"])
df_group["order_date_shifted"] = df_group.groupby("user_id")["order_date"].shift(1)
# df_group[['A', 'B', 'C']] = df_gr.groupby('Name')['A', 'B', 'C'].shift(1)
df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group.fillna(0, inplace=True)
df_group

,order_date,user_id,order_date_shifted,days_since_last_order
24723,2012-04-19,6,NaT,NaN
31011,2012-04-25,6,2012-04-19,6.0
226242,2012-09-23,6,2012-04-25,151.0
240276,2012-10-06,6,2012-09-23,13.0
331324,2013-01-11,6,2012-10-06,97.0
...,...,...,...,...
531141,2013-04-29,91911,NaT,NaN
531149,2013-04-27,91912,NaT,NaN
531154,2013-04-29,91915,NaT,NaN
531158,2013-04-29,91920,NaT,NaN


In [152]:
cols_to_agg = ["user_id"]
stats_to_agg = ["mean","median","max","min","std","mad"]
# Loop through the columns to aggregate
for i in cols_to_agg:
    df_group_group = df_group[[i,"days_since_last_order"]].groupby([i]).agg(stats_to_agg).reset_index().round(2)
    df_group_group.columns = [str("_" + i +"_").join(col) for col in df_group_group.columns]
    df_group_group.rename(columns= {i+"_"+i+"_":i},inplace=True)
    df_group = df_group.merge(df_group_group,on=i,how="left")
df_group.fillna(0,inplace=True)
df_group.drop(["order_date_shifted"],axis=1,inplace=True)
df = df.merge(df_group,on=["user_id","order_date"],how="left")    

In [19]:
# df_group = df[["order_date","user_id","size"]].drop_duplicates().sort_values(by=["user_id","order_date"])
# df_group["order_date_shifted"] = df_group.groupby(["user_id","size"])["order_date"].shift(1)
# # df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# # df_group.fillna(0, inplace=True)
# df_group["days_since_last_order"+ "_size"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group[df_group["user_id"]== 6] 

,order_date,user_id,size,order_date_shifted,days_since_last_order_size
24723,2012-04-19,6,s,NaT,NaN
31011,2012-04-25,6,s,2012-04-19,6.0
226242,2012-09-23,6,s,2012-04-25,151.0
226244,2012-09-23,6,xs,NaT,NaN
240276,2012-10-06,6,s,2012-09-23,13.0
240277,2012-10-06,6,xs,2012-09-23,13.0
240280,2012-10-06,6,m,NaT,NaN
240282,2012-10-06,6,unsized,NaT,NaN
331324,2013-01-11,6,s,2012-10-06,97.0
331329,2013-01-11,6,unsized,2012-10-06,97.0


In [ ]:
# df_group = df[["order_date","user_id","size"]].drop_duplicates().sort_values(by=["user_id","order_date"])
# df_group["order_date_shifted"] = df_group.groupby(["user_id","size"])["order_date"].shift(1)
# # df_group["days_since_last_order"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# # df_group.fillna(0, inplace=True)
# df_group["days_since_last_order"+ "_size"] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
# df_group[df_group["user_id"]== 6] 

In [157]:
item_descriptions = ["item_id","size","item_color","brand_id"]
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))

In [158]:
categorical_powerset = powerset(item_descriptions)
# Use the powerset to create new columns indicating the days since the last order for each combination of the item descriptions
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id as they are redundant
exclude = ["item_id","brand_id"]
for column in categorical_powerset:
    current = list(column)
    if not all(x in current for x in exclude):
        print(current)
        df_group = df[["order_date","user_id"]+current].drop_duplicates().sort_values(by=["user_id","order_date"])
        df_group["order_date_shifted"] = df_group.groupby(["user_id"] + current)["order_date"].shift(1)
        df_group["days_since_last_order_same"+ "_" + "_".join(current)] = (df_group["order_date"] - df_group["order_date_shifted"]).dt.days
        df_group.fillna(0, inplace=True)
        df_group.drop(["order_date_shifted"],axis=1,inplace=True)
        df = df.merge(df_group,on=["user_id","order_date"]+current,how="left")

['item_id']
['size']
['item_color']
['brand_id']
['item_id', 'size']
['item_id', 'item_color']
['size', 'item_color']
['size', 'brand_id']
['item_color', 'brand_id']
['item_id', 'size', 'item_color']
['size', 'item_color', 'brand_id']


In [35]:
# Instantiate the model class
Model = Model_class(df)

In [36]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)

In [115]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,1,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu)

Catboost_cpu MAE on test set: 0.3166


In [114]:
catboost_params_gpu = {'iterations': 250, 'learning_rate': 0.10, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

Catboost_gpu MAE on test set: 0.317


In [40]:
Model.get_feature_importance(catboost_model_cpu,df_train)[:10]

[('delivery_time', 16.45957176390631),
 ('user_id', 12.63236918304526),
 ('delivery_weekday', 8.798324056961754),
 ('item_id', 7.032381298865096),
 ('order_sum', 3.430829502910217),
 ('order_item_id_size_nunique', 1.9171655546995419),
 ('delivery_time_user_id_median', 1.7392177398791335),
 ('days_since_last_order_same_item_id', 1.523784869650899),
 ('delivery_time_user_id_min', 1.4402357517386228),
 ('order_number_same_item_id_item_color', 1.3121296999226364)]

In [113]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

Xgboost MAE on test set: 0.3236


In [11]:
mae_dict_1 = {"Catboost_native_CPU":0.3166,"Catboost_native_GPU":0.317,"Xgboost":0.3236}
model_results_1 = pd.DataFrame(mae_dict_1,index=["MAE_4"])
model_results_1.sort_values(by="MAE_4",ascending=True,inplace=True,axis=1)
model_results_1 = model_results_1.transpose()
model_results_1.index.set_names(["Model"],inplace=True)
model_results_1.reset_index(inplace=True)
model_results_1

,Model,MAE_4
0,Catboost_native_CPU,0.3166
1,Catboost_native_GPU,0.3170
2,Xgboost,0.3236


In [12]:
model_performance = pd.read_csv("Model_Performances/Model_performance_1.csv")
model_performance

,Model,MAE_3
0,Catboost_native_GPU,0.3191
1,Catboost_native_CPU,0.3196
2,Xgboost,0.3258
3,Lightgbm_native,0.3301


In [18]:
model_results_whole = pd.merge(model_results_1,model_performance,on="Model",how="left")
model_results_whole["MAE_Improvement"] = model_results_whole["MAE_3"] - model_results_whole["MAE_4"]
model_results_whole = model_results_whole[["Model","MAE_3","MAE_4","MAE_Improvement"]]
model_results_whole

,Model,MAE_3,MAE_4,MAE_Improvement
0,Catboost_native_CPU,0.3196,0.3166,0.0030
1,Catboost_native_GPU,0.3191,0.3170,0.0021
2,Xgboost,0.3258,0.3236,0.0022


In [19]:
# df["order_date"] = pd.to_datetime(df["order_date"], format="%Y-%m-%d")
# df["delivery_date"] = pd.to_datetime(df["delivery_date"], format="%Y-%m-%d")

# # Get the year for order date and delivery date
# df['order_year'] = df['order_date'].dt.year
# df['delivery_year'] = df['delivery_date'].dt.year

# # Get the season for order date and delivery date
# df['order_season'] = df['order_date'].dt.month.apply(lambda x: 'winter' if x in [12,1,2] else 'spring' if x in [3,4,5] else 'summer' if x in [6,7,8] else 'autumn')
# df['delivery_season'] = df['delivery_date'].dt.month.apply(lambda x: 'winter' if x in [12,1,2] else 'spring' if x in [3,4,5] else 'summer' if x in [6,7,8] else 'autumn')

# # Get the day of the year for order date and delivery date
# df['order_day_of_year'] = df['order_date'].dt.dayofyear
# df['delivery_day_of_year'] = df['delivery_date'].dt.dayofyear

In [179]:
interaction_cols = cat_cols.copy()
interaction_cols += ["order_weekday","delivery_weekday","delivery_week","order_item_count","delivery_time"]
# Remove the columns that are not needed for the interaction
interaction_cols = [x for x in interaction_cols if x not in ["mode_item_id","mode_size","mode_brand_id","mode_item_color","user_state","user_title"]]
interaction_cols

['size',
 'item_color',
 'item_id',
 'brand_id',
 'user_id',
 'order_weekday',
 'delivery_weekday',
 'delivery_week',
 'order_item_count',
 'delivery_time']

In [ ]:
# Delivery time, order sum, user_age, item price binning

In [180]:
# !! Include in code base
# Use the numeric variable order_sum to create a new categorical variable using bins
bins_range = df["order_sum"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9])
# bins_range
bins = bins_range[["min","10%","25%","50%","75%","90%","max"]].values
df['order_sum_bins'] = pd.cut(df['order_sum'], bins=bins, include_lowest=True).cat.codes

In [181]:
bins_range = df["item_price"].describe(percentiles=[0.1,0.25,0.5,0.75,0.95])
bins = bins_range[["min","10%","25%","50%","75%","95%","max"]].values
df['item_price_bins'] = pd.cut(df['item_price'], bins=bins, include_lowest=True).cat.codes

In [ ]:
# catboost_cpu_importance_list = [col for tuple in catboost_cpu_importance[:50] for col in tuple]
# # code base convert list of tuples to list
# # out = [item for t in listoftuples for item in t]
# catboost_cpu_importance_list

In [101]:
# df.loc[:, 'order_number_same_item_id':'order_brand_id_item_id_nunique'].columns
# order_number_important_list = []
# for col in df.loc[:, 'order_number_same_item_id':'order_brand_id_item_id_nunique'].columns:
#     if col in catboost_cpu_importance_list:
#         order_number_important_list.append(col)
# order_number_important_list.remove("order_number_different_size_item_color_brand_id")
# order_number_important_list.remove("order_item_id_size_nunique")
# order_number_important_list

['order_number_same_size',
 'order_number_same_item_id_size',
 'order_number_different_item_id_size',
 'order_number_same_item_id_item_color']

In [182]:
order_number_important_list =['order_number_same_size','order_number_same_item_id_size','order_number_different_item_id_size','order_number_same_item_id_item_color']
interaction_cols += order_number_important_list
interaction_cols += ["order_sum_bins","item_price_bins"]


In [183]:
len(interaction_cols)

16

In [185]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(2,3))
interaction_powerset = powerset(interaction_cols)

In [186]:
# Use the powerset to create new columns with the interaction between the categorical variables
powerset_name_list = []
# exclude powerset elements containing both item_id and brand_id
exclude = ["item_id","brand_id"]
for column in interaction_powerset:
    current = list(column)
    print(current)
    column_name = ""
    # sum the strings in the column
    column_name = "_x_".join(current)
    # df[column_name] = df[current].astype(str).sum(axis=1)
    df[column_name] = df[current[0]].astype(str) + "_" + df[current[1]].astype(str)
    df[column_name] = LE.fit_transform(df[column_name])
    # powerset_name_list.append(column_name)

['size', 'item_color']
['size', 'item_id']
['size', 'brand_id']
['size', 'user_id']
['size', 'order_weekday']
['size', 'delivery_weekday']
['size', 'delivery_week']
['size', 'order_item_count']
['size', 'delivery_time']
['size', 'order_number_same_size']
['size', 'order_number_same_item_id_size']
['size', 'order_number_different_item_id_size']
['size', 'order_number_same_item_id_item_color']
['size', 'order_sum_bins']
['size', 'item_price_bins']
['item_color', 'item_id']
['item_color', 'brand_id']
['item_color', 'user_id']
['item_color', 'order_weekday']
['item_color', 'delivery_weekday']
['item_color', 'delivery_week']
['item_color', 'order_item_count']
['item_color', 'delivery_time']
['item_color', 'order_number_same_size']
['item_color', 'order_number_same_item_id_size']
['item_color', 'order_number_different_item_id_size']
['item_color', 'order_number_same_item_id_item_color']
['item_color', 'order_sum_bins']
['item_color', 'item_price_bins']
['item_id', 'brand_id']
['item_id', 'us

In [ ]:
to_drop =["item_id_x_brand_id","order_sum_bins","item_price_bins"]
df.drop(to_drop, axis=1, inplace=True)
df

In [189]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531170 entries, 0 to 531169
Columns: 264 entries, order_item_id to delivery_time_x_item_price_bins
dtypes: category(1), datetime64[ns](4), float64(72), int32(125), int64(62)
memory usage: 833.3 MB


In [191]:
df_interactions = df.loc[:, "size_x_item_color":].columns


In [193]:
cat_cols += list(df_interactions)

In [58]:
# cat_cols += ['order_number_same_item_id_item_color_x_item_price_bins', 'order_sum_bins_x_item_price_bins']

In [194]:
# Instantiate the model class
Model = Model_class(df)

In [195]:
# Split the data into train and test sets
df_train,df_test, df_valid = Model.split_data(False)

In [201]:
# Encode the categorical variables using leave-one-out encoding
df_train_encoded,df_test_encoded,encoder = Model.LOE_Encoder(df_train,df_test, cat_cols,0.1,True)

In [198]:
catboost_params_gpu = {'iterations': 100, 'learning_rate': 0.10, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42,"task_type":"GPU"}
catboost_model_gpu, catboost_Y_pred_gpu, catboost_mae_gpu, catboost_Y_pred_proba_gpu = Model.catboost(df_train,df_test,catboost_params_gpu,1,cat_cols)
print("Catboost_gpu MAE on test set:", catboost_mae_gpu)

0:	learn: 0.6720839	test: 0.6767233	best: 0.6767233 (0)	total: 487ms	remaining: 48.2s
1:	learn: 0.6551537	test: 0.6632787	best: 0.6632787 (1)	total: 892ms	remaining: 43.7s
2:	learn: 0.6414416	test: 0.6526650	best: 0.6526650 (2)	total: 1.32s	remaining: 42.6s
3:	learn: 0.6305226	test: 0.6437582	best: 0.6437582 (3)	total: 1.78s	remaining: 42.6s
4:	learn: 0.6206886	test: 0.6367847	best: 0.6367847 (4)	total: 2.2s	remaining: 41.8s
5:	learn: 0.6127217	test: 0.6311212	best: 0.6311212 (5)	total: 2.62s	remaining: 41.1s
6:	learn: 0.6060471	test: 0.6263331	best: 0.6263331 (6)	total: 3.04s	remaining: 40.4s
7:	learn: 0.6009599	test: 0.6227677	best: 0.6227677 (7)	total: 3.47s	remaining: 39.9s
8:	learn: 0.5965978	test: 0.6197281	best: 0.6197281 (8)	total: 3.88s	remaining: 39.2s
9:	learn: 0.5928400	test: 0.6170716	best: 0.6170716 (9)	total: 4.28s	remaining: 38.5s
10:	learn: 0.5893564	test: 0.6152502	best: 0.6152502 (10)	total: 4.68s	remaining: 37.9s
11:	learn: 0.5863009	test: 0.6133542	best: 0.6133542 

In [204]:
top_100 = Model.get_feature_importance(catboost_model_gpu,df_train)[0:100]


[('user_id', 10.772475374549618),
 ('user_id_x_delivery_weekday', 6.019343009166123),
 ('user_id_x_delivery_time', 5.4434551467051175),
 ('delivery_weekday_x_delivery_week', 5.405200002816554),
 ('delivery_time_x_item_price_bins', 4.352488719328298),
 ('item_id_x_user_id', 4.235113001787971),
 ('item_id_x_order_number_same_item_id_item_color', 3.7662544266648053),
 ('days_since_last_order_same_item_id', 3.736171785564035),
 ('delivery_time_x_order_sum_bins', 2.4536087538852396),
 ('item_id_x_item_price_bins', 2.237419527847821),
 ('order_weekday_x_delivery_weekday', 2.1853607348259825),
 ('size_x_user_id', 2.0801977256189934),
 ('item_color_x_item_id', 1.8555123234932853),
 ('delivery_weekday_x_delivery_time', 1.8463076027443757),
 ('user_id_x_order_number_different_item_id_size', 1.788844813382942),
 ('delivery_time_x_order_number_same_item_id_item_color', 1.7816920768979436),
 ('order_item_id_size_nunique', 1.6925615957418638),
 ('delivery_weekday_x_order_item_count', 1.6835131933152

In [213]:
top_100_list = [col for tuple in top_100 for col in tuple if isinstance(col, str)]
top_100_list
# for i in top_100:
#     top_100_list.append(i[0])
#     print(i[0])

['user_id',
 'user_id_x_delivery_weekday',
 'user_id_x_delivery_time',
 'delivery_weekday_x_delivery_week',
 'delivery_time_x_item_price_bins',
 'item_id_x_user_id',
 'item_id_x_order_number_same_item_id_item_color',
 'days_since_last_order_same_item_id',
 'delivery_time_x_order_sum_bins',
 'item_id_x_item_price_bins',
 'order_weekday_x_delivery_weekday',
 'size_x_user_id',
 'item_color_x_item_id',
 'delivery_weekday_x_delivery_time',
 'user_id_x_order_number_different_item_id_size',
 'delivery_time_x_order_number_same_item_id_item_color',
 'order_item_id_size_nunique',
 'delivery_weekday_x_order_item_count',
 'delivery_time_x_order_number_same_item_id_size',
 'order_number_same_item_id',
 'item_price_user_id_sum',
 'item_id',
 'delivery_week_x_delivery_time',
 'user_id_x_order_number_same_size',
 'order_number_same_item_id_item_color_x_order_sum_bins',
 'brand_id_x_order_sum_bins',
 'delivery_time_user_id_min',
 'item_price_user_id_count',
 'order_weekday_x_delivery_time',
 'size_x_it

In [ ]:
interactions_importance = Model.feature_importance(catboost_model_gpu, cat_cols)

In [ ]:
catboost_params_cpu = {'iterations': 150, 'learning_rate': 0.1, 'depth': 10, 'loss_function': 'Logloss', 'random_seed': 42}
catboost_model_cpu, catboost_Y_pred_cpu, catboost_mae_cpu, catboost_Y_pred_proba_cpu = Model.catboost(df_train,df_test,catboost_params_cpu,1,cat_cols)
print("Catboost_cpu MAE on test set:", catboost_mae_cpu)

In [61]:
Model.get_feature_importance(catboost_model_cpu,df_train)

[('order_weekday_x_delivery_weekday', 11.201640533967337),
 ('user_id', 8.230001780102228),
 ('user_id_x_delivery_weekday', 5.720732678615244),
 ('item_id_x_user_id', 4.022871213911953),
 ('delivery_weekday_x_delivery_week', 3.788687216467598),
 ('item_id_x_order_number_same_item_id_item_color', 2.6982403753984636),
 ('delivery_time', 2.258498084497127),
 ('user_id_x_delivery_week', 2.0112232089572997),
 ('item_id_x_item_price_bins', 1.894513848426602),
 ('size_x_user_id', 1.8791192914508645),
 ('user_id_x_order_number_different_item_id_size', 1.839668768234921),
 ('days_since_last_order_same_item_id', 1.814049277174382),
 ('item_color_x_item_id', 1.3987060267641322),
 ('user_id_x_order_item_count', 1.171946088899031),
 ('delivery_weekday_x_item_price_bins', 1.119149239403767),
 ('brand_id_x_order_sum_bins', 1.0970783268453632),
 ('order_item_id_size_nunique', 1.0640502222907942),
 ('size_x_item_id', 1.011583063356824),
 ('item_color_x_user_id', 0.9311620673935717),
 ('item_id_x_order_

In [202]:
xgboost_params = {'max_depth': 9, 'learning_rate': 0.15, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 42,"objective":"binary:logistic"}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_proba = Model.xgboost(df_train_encoded,df_test_encoded,xgboost_params,1)
print("Xgboost MAE on test set:", xgb_mae)

[0]	validation_0-mae:0.49041
[1]	validation_0-mae:0.48243
[2]	validation_0-mae:0.47566
[3]	validation_0-mae:0.46939
[4]	validation_0-mae:0.46403
[5]	validation_0-mae:0.45949
[6]	validation_0-mae:0.45578
[7]	validation_0-mae:0.45277
[8]	validation_0-mae:0.45006
[9]	validation_0-mae:0.44751
[10]	validation_0-mae:0.44517
[11]	validation_0-mae:0.44348
[12]	validation_0-mae:0.44220
[13]	validation_0-mae:0.44036
[14]	validation_0-mae:0.43892
[15]	validation_0-mae:0.43721
[16]	validation_0-mae:0.43606
[17]	validation_0-mae:0.43515
[18]	validation_0-mae:0.43395
[19]	validation_0-mae:0.43331
[20]	validation_0-mae:0.43242
[21]	validation_0-mae:0.43180
[22]	validation_0-mae:0.43079
[23]	validation_0-mae:0.43008
[24]	validation_0-mae:0.42965
[25]	validation_0-mae:0.42933
[26]	validation_0-mae:0.42871
[27]	validation_0-mae:0.42835
[28]	validation_0-mae:0.42799
[29]	validation_0-mae:0.42768
[30]	validation_0-mae:0.42686
[31]	validation_0-mae:0.42632
[32]	validation_0-mae:0.42632
[33]	validation_0-ma

In [203]:
Model.get_feature_importance(xgb_model,df_train_encoded)

[('user_id_x_delivery_time_LOE_encoded', 0.07900429),
 ('item_id_x_order_sum_bins_LOE_encoded', 0.046823744),
 ('user_id_x_delivery_weekday_LOE_encoded', 0.046650458),
 ('delivery_time_x_item_price_bins_LOE_encoded', 0.0427944),
 ('order_number_same_item_id_item_color', 0.025408747),
 ('user_id_x_order_number_same_item_id_size_LOE_encoded', 0.019417152),
 ('item_id_x_order_number_same_item_id_item_color_LOE_encoded', 0.017232347),
 ('user_id_LOE_encoded', 0.015677378),
 ('order_number_different_item_id_size_item_color', 0.014801187),
 ('user_id_x_order_number_different_item_id_size_LOE_encoded', 0.013504232),
 ('delivery_weekday_x_delivery_time_LOE_encoded', 0.012400237),
 ('order_item_count', 0.012199569),
 ('order_item_id_size_nunique', 0.010938023),
 ('delivery_weekday_x_delivery_week_LOE_encoded', 0.010710115),
 ('item_id_x_delivery_weekday_LOE_encoded', 0.010326452),
 ('item_id_x_delivery_time_LOE_encoded', 0.010128442),
 ('order_number_same_item_id_size', 0.009940118),
 ('item_id